In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [5]:
df_ff_data = pd.read_csv(
    "DATA/CSV DATA/2021.csv")
df_ff_data.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,...,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,...,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,...,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,...,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,...,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [6]:
pos_mapping = {"WR": "WR", "RB": "RB", "QB": 'QB', "TE": 'TE'}
df_ff_data["POS_encoded"] = df_ff_data["Pos"].map(pos_mapping)
df_ff_data.dropna(subset=["POS_encoded"], inplace=True)
#df_ff_data["POS_encoded"] = df_ff_data["POS_encoded"].astype(int)
df_ff_data

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,...,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,POS_encoded
0,0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,...,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0,RB
1,1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,...,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0,WR
2,2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,...,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0,WR
3,3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,...,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0,QB
4,4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,...,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,665,Logan Woodside,TEN,QB,26,5,0,0.0,0.0,0.0,...,-6.0,0.0,6.0,0.0,0.0,-1.0,0.0,0.0,0.0,QB
667,667,Trenton Cannon,2TM,RB,27,12,0,0.0,0.0,0.0,...,4.0,0.0,3.0,0.0,0.0,-2.0,0.0,1.0,1.0,RB
668,668,John Wolford,LAR,QB,26,3,0,0.0,0.0,5.0,...,-1.0,0.0,2.0,0.0,0.0,-2.0,1.0,0.0,0.0,QB
669,669,Josh Rosen,ATL,QB,24,4,0,0.0,0.0,19.0,...,0.0,0.0,0.0,0.0,0.0,-3.0,2.0,0.0,0.0,QB


In [7]:
df_ff_data = df_ff_data.drop(columns=["Unnamed: 0", 'Tm',"Player","Pos"])
df_ff_data.head()

,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,POS_encoded
0,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0,RB
1,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0,WR
2,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0,WR
3,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0,QB
4,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0,RB


In [8]:
y = df_ff_data["POS_encoded"]
x = df_ff_data.drop(columns = "POS_encoded" )
x.fillna(x.mean(), inplace=True)

In [9]:
len(y)

596

In [10]:
len(x)

596

In [11]:
balance = y.value_counts()
balance

POS_encoded
WR    227
RB    168
TE    118
QB     83
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier
classifier.fit(X_train, y_train)

C:\Users\EvanS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [14]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
13,RB,RB
372,WR,WR
117,WR,WR
547,RB,RB
19,QB,QB
...,...,...
329,WR,WR
583,QB,QB
180,TE,TE
282,RB,RB


In [15]:
balanced_accuracy_score(y_test, predictions)

0.8809091694754126

In [16]:
confusion_matrix(y_test, predictions)

array([[21,  0,  0,  0],
       [ 1, 38,  1,  2],
       [ 0,  0, 21,  8],
       [ 1,  0,  5, 51]], dtype=int64)

In [17]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

          QB       0.91      1.00      0.95        21
          RB       1.00      0.90      0.95        42
          TE       0.78      0.72      0.75        29
          WR       0.84      0.89      0.86        57

    accuracy                           0.88       149
   macro avg       0.88      0.88      0.88       149
weighted avg       0.88      0.88      0.88       149

